In [1]:
import numpy as np
import os
from pathlib import Path
from PIL import Image
import json
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import transforms as T
import torch
import random
import torch.nn as nn
from torch.utils.data import Dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from CellDataset import CellDataset, moco_transform
from MoCoResNetBackbone import MoCoResNetBackbone

In [2]:
modelPath = Path("/scratch/cv-course-group-5/models/training4/model_epoch50.pth")

gpu = 0

device = torch.device(f"cuda:{gpu}" if torch.cuda.is_available() else "cpu")

model = MoCoResNetBackbone()

model_state_dict = torch.load(modelPath, map_location=device)
model.load_state_dict(model_state_dict)
model.eval()
model.to(device)

with open(Path('train_test_split.json'), 'r') as f:
    _split_data = json.load(f)
val_list = _split_data.get("val", [])
dataset = CellDataset(video_list=val_list[:5], mode='inference')

22447


In [3]:
batch_size = 128
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=8, pin_memory=True)

all_embeddings = []
all_labels = []

In [4]:
with torch.no_grad():
    for imgs, labels in tqdm(dataloader, desc="Extracting embeddings"):
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.cpu().numpy()  # optional, if tensor
        embeddings = model.encode_query(imgs)  # → (B, 2048)
        embeddings = embeddings.cpu().numpy()

        all_embeddings.append(embeddings)
        all_labels.append(labels)

embeddings = np.concatenate(all_embeddings, axis=0)
labels = np.concatenate(all_labels, axis=0)

Extracting embeddings: 100%|███████████████████████████████████████████| 176/176 [00:16<00:00, 10.92it/s]


In [5]:
class EmbeddingDataset(Dataset):
    def __init__(self, data_array: np.ndarray, label_array: np.ndarray):
        assert data_array.shape[0] == label_array.shape[0], "Mismatched data and labels"
        self.data = torch.from_numpy(data_array).float()
        self.labels = torch.from_numpy(label_array).byte()  # Use .float() if labels are floats

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [6]:

train_dataset = EmbeddingDataset(data_array=embeddings[:int(0.8 * len(embeddings))], label_array=labels[:int(0.8 * len(embeddings))])
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)

val_dataset = EmbeddingDataset(data_array=embeddings[int(0.8 * len(embeddings)):], label_array=labels[int(0.8 * len(embeddings)):])
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)

In [7]:
_, train_labels = train_dataset[:]

linear_model = nn.Linear(embeddings.shape[1], 1)

epochs = 5
learning_rate = 0.001

pos_weight = (len(train_labels) - train_labels.sum()) / train_labels.sum()
loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = torch.optim.Adam(linear_model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    epoch_loss = []
    linear_model.train()
    for embedding, label in tqdm(train_dataloader, desc=f"Epoch {epoch}", total=len(train_dataloader), ncols=100):
        pred = linear_model(embedding)
        loss = loss_fn(pred, label.unsqueeze(1).float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss.append(loss.item())
    print(f"Train Loss Epoch {epoch}: {np.mean(epoch_loss)}", end="")
    linear_model.eval()
    val_loss = []
    for embedding, label in val_dataloader:
        pred = (torch.sigmoid(linear_model(embedding)) > 0.5).squeeze().int()
        val_loss.append(np.sum([label != pred]) / batch_size)

    print(f"Val Loss Epoch {epoch}: {np.mean(val_loss)}")


Epoch 0: 100%|███████████████████████████████████████████████████| 141/141 [00:01<00:00, 130.32it/s]

Train Loss Epoch 0: 0.03527384174076167

Val Loss Epoch 0: 0.4264322916666667


Epoch 1: 100%|███████████████████████████████████████████████████| 141/141 [00:00<00:00, 147.85it/s]

Train Loss Epoch 1: 0.0318620673162823

...

Train Loss Epoch 3: 0.03149951854390456

Val Loss Epoch 3: 0.23958333333333334


Epoch 4: 100%|████████████████████████████████████████████████████| 141/141 [00:01<00:00, 95.80it/s]

Train Loss Epoch 4: 0.03011958591032324

Val Loss Epoch 4: 0.14583333333333334


In [8]:
random_forest = RandomForestClassifier(n_estimators=500, criterion="entropy", class_weight="balanced", n_jobs=8, random_state=42)

embeddings_train, labels_train = train_dataset[:]
embeddings_val, labels_val = val_dataset[:]

random_forest.fit(embeddings_train, labels_train)

KeyboardInterrupt: 

In [9]:
pred_val = random_forest.predict(embeddings_val)

print(classification_report(labels_val, pred_val))

IndexError: list index out of range

In [14]:
pca = PCA(256)
embeddings_train_reduced = pca.fit_transform(embeddings_train)
embeddings_val_reduced = pca.transform(embeddings_val)

In [15]:
svm = SVC(kernel="rbf", probability=True, class_weight="balanced", random_state=42)

svm.fit(embeddings_train_reduced, labels_train)

pred_val = svm.predict(embeddings_val_reduced)

print(classification_report(labels_val, pred_val))

              precision    recall  f1-score   support

           0       0.09      0.86      0.16       161
           1       0.99      0.67      0.80      4329

    accuracy                           0.68      4490
   macro avg       0.54      0.77      0.48      4490
weighted avg       0.96      0.68      0.78      4490

